In [1]:
import tensorflow as tf
from tensorflow import keras
import PIL
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import numpy as np
import json
from tensorflow.keras.models import load_model
from tensorflow.keras.models import model_from_json
from tensorflow.keras.preprocessing import image

from tensorflow.keras import Sequential

In [2]:
image_size = (224,224)

In [3]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
        'Fire-vs-NoFire\\Training',image_size=image_size ,validation_split=0.2, subset="validation", seed=1337,
        batch_size=32, shuffle=True
        )

Found 39375 files belonging to 2 classes.
Using 7875 files for validation.


In [5]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
        'Fire-vs-NoFire\\Training',image_size=image_size, validation_split=0.2, subset="training",seed=1337,batch_size=32,shuffle=True
)

Found 39375 files belonging to 2 classes.
Using 31500 files for training.


In [9]:
def make_model_keras(input_shape, num_classes):

    print(input_shape)
   
    inputs = keras.Input(shape=input_shape)
    
    x = inputs  
    x = layers.Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu")(x)
   #x = layers.Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu")(x)
    x = layers.MaxPooling2D(3,strides=2)(x)
    x = layers.Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu")(x)
   # x = layers.Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu")(x)
    x = layers.MaxPooling2D(3,strides=2)(x)
    x = layers.Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu")(x)
  #  x = layers.Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu")(x)
    #x = layers.Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu")(x)
    x = layers.MaxPooling2D(3,strides=2)(x)
    x = layers.Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(x)
  #  x = layers.Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(x)
  #  x = layers.Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(x)
    x = layers.MaxPooling2D(2,strides=2)(x)
    x = layers.Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(x)
   # x = layers.Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(x)
   # x = layers.Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(x)
    x = layers.MaxPooling2D(3,strides=2)(x)   
    x = layers.Flatten()(x)
    x = layers.Dense(units=4096,activation="relu")(x)
   # x = layers.Dense(units=4096,activation="relu")(x)

    
    if num_classes == 2:
        activation = "sigmoid"
        units = 1
    else:
        activation = "softmax"
        units = num_classes
    
    
    
    outputs = layers.Dense(units, activation=activation)(x)
    return keras.Model(inputs, outputs, name="model_vgg")


    


In [10]:
train_ds = train_ds.prefetch(buffer_size=32)
val_ds = val_ds.prefetch(buffer_size=32)

model = make_model_keras(input_shape=image_size + (3,), num_classes=2)
#model.summary
model.summary()
#keras.utils.plot_model(model, show_shapes=True)

(224, 224, 3)
Model: "model_vgg"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 224, 224, 64)      1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 111, 111, 64)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 111, 111, 128)     73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 55, 55, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 55, 55, 256)       295168    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 27, 27,

In [11]:
callbacks = [keras.callbacks.ModelCheckpoint(filepath="VGG_MODEL\\save_at_{epoch}.h5", verbose=1, save_best_only=True), ]
#otp = keras.optimizers.
model.compile(optimizer=keras.optimizers.SGD(
    learning_rate=0.01, momentum=0.9), loss="binary_crossentropy", metrics=["accuracy"], )
model.fit(train_ds, epochs=5, callbacks=callbacks, validation_data=val_ds, batch_size=32)

Epoch 1/5
985/985 [==============================] - ETA: 0s - loss: nan - accuracy: 0.6342
Epoch 00001: val_loss did not improve from inf
985/985 [==============================] - 2694s 3s/step - loss: nan - accuracy: 0.6342 - val_loss: nan - val_accuracy: 0.6370
Epoch 2/5
985/985 [==============================] - ETA: 0s - loss: nan - accuracy: 0.6350
Epoch 00002: val_loss did not improve from inf
985/985 [==============================] - 2581s 3s/step - loss: nan - accuracy: 0.6350 - val_loss: nan - val_accuracy: 0.6370
Epoch 3/5
985/985 [==============================] - ETA: 0s - loss: nan - accuracy: 0.6350
Epoch 00003: val_loss did not improve from inf
985/985 [==============================] - 2706s 3s/step - loss: nan - accuracy: 0.6350 - val_loss: nan - val_accuracy: 0.6370
Epoch 4/5
985/985 [==============================] - ETA: 0s - loss: nan - accuracy: 0.6350
Epoch 00004: val_loss did not improve from inf
985/985 [==============================] - 2637s 3s/step - loss

In [13]:
 model.save("VGG16_MODEL\\model")

INFO:tensorflow:Assets written to: VGG16_MODEL\model\assets
